In [1]:
from pymilvus import MilvusClient
from pymilvus import CollectionSchema, FieldSchema, DataType


import pandas as pd
from tqdm import tqdm
import numpy as np
from typing import Dict , List, Any
import time
import json



from FlagEmbedding import BGEM3FlagModel
embeddings_dim = 1024


### Create Collection

In [92]:
client.list_collections()

['articles_collection_L2',
 'articles_collection_IP',
 'articles_collection_COSINE',
 'articles_collectionPartition_L2',
 'articles_collectionPartition_IP',
 'articles_collectionPartition_COSINE',
 'Schunk_articles_collection',
 'articles_collection']

In [3]:
# client.drop_collection("RSchunk_articles_collection")

In [2]:
client = MilvusClient(uri="http://localhost:19530")

In [4]:

collection_name = 'RSchunk_articles_collection'

id_field = FieldSchema(name="id", dtype=DataType.INT64, is_primary=True)
id_article_field = FieldSchema(name="article_id", dtype=DataType.INT64)
text_field = FieldSchema(name="chunk_article", dtype=DataType.VARCHAR, max_length=65535)
reference_field = FieldSchema(name="reference", dtype=DataType.VARCHAR, max_length=1000)
embedding_field = FieldSchema(name="embedding_articles", dtype=DataType.FLOAT16_VECTOR, dim=embeddings_dim)

schema = CollectionSchema(fields=[id_field, id_article_field, text_field, reference_field, embedding_field], description="collection d'articles de loi")

client.create_collection(collection_name = collection_name, schema=schema)

##### Prep Indexing

In [5]:
#  set the indexing
index_params = MilvusClient.prepare_index_params()

# Add an index on the vector field.
index_params.add_index(
    field_name="embedding_articles",
    metric_type="COSINE",
    index_type="FLAT",
    index_name="vector_index",
    params={}   #128 clusters units, 8 cluster search
)

client.create_index(
    collection_name=collection_name,
    index_params=index_params,
    sync=False # Whether to wait for index creation to complete before returning. Defaults to True.
)


### Create Partition and Insert entities

##### Create Partitions

In [6]:
# load data
# df_chunk_articles = pd.read_csv('chunked_articles.csv')
df_Recursivechunked_articles = pd.read_csv('Recursivechunked_articles.csv')

In [7]:
# length of the chunk
def len_chunk(chunk):
    return len(chunk.split())

df_Recursivechunked_articles['chunk'] = df_Recursivechunked_articles['chunk'].astype(str)
df_Recursivechunked_articles['len_chunk'] = df_Recursivechunked_articles['chunk'].apply(len_chunk)

In [8]:

df_Recursivechunked_articles = df_Recursivechunked_articles[df_Recursivechunked_articles['len_chunk'] > 10]
# rename the column "Unnamed: 0" to id_chunk
df_Recursivechunked_articles.rename(columns={'Unnamed: 0': 'id_chunk'}, inplace=True)
df_Recursivechunked_articles.head(5)

,id_chunk,article_id,code,reference,chunk,len_chunk
740,740,5311,Code Judiciaire,"Art. 1202, Code Judiciaire (Livre IV, Chapitre...",Il ne peut faire l'avance au vendeur du prix n...,11
741,741,17480,Code de la Démocratie Locale et de la Décentra...,"Art. L2212-19, Code de la Démocratie Locale et...",§ 1er La séance est ouverte et close par le pr...,11
742,742,15350,Code de Droit Economique,"Art. XII.24, Code de Droit Economique (Livre X...",§ 1er Le présent titre met en oeuvre le règlem...,11
743,743,4834,Code Judiciaire,"Art. 745, Code Judiciaire (Livre II, Titre II,...","20), abrogé lui-même par l'art 176, 10° de L20...",11
744,744,17474,Code de la Démocratie Locale et de la Décentra...,"Art. L2212-13, Code de la Démocratie Locale et...",Chaque groupe politique désigne en son sein un...,11


In [9]:
df_chunk_articles =  df_Recursivechunked_articles.copy()

In [10]:
import re

def normalize_partition_name(name):
    # Replace the spaces and dashes with underscores
    name = re.sub(r'\s+|-', '_', name)
    # remove non-alphanumeric characters
    name = re.sub(r'[^a-zA-Z0-9_]', '', name)
    return name

# Appliquer cette fonction aux noms des codes
df_chunk_articles['normalized_cat'] = df_chunk_articles['code'].apply(normalize_partition_name)


In [11]:
codes

array(['Code_Bruxellois_de_lAir_du_Climat_et_de_la_Matrise_de_lEnergie',
       'Code_Bruxellois_de_lAmnagement_du_Territoire',
       'Code_Bruxellois_du_Logement', 'Code_Civil', 'Code_Consulaire',
       'Code_Electoral', 'Code_Electoral_Communal_Bruxellois',
       'Code_Ferroviaire', 'Code_Forestier', 'Code_Judiciaire',
       'Code_Pnal', 'Code_Pnal_Militaire', 'Code_Pnal_Social',
       'Code_Rural',
       'Code_Rglementaire_Wallon_de_lAction_sociale_et_de_la_Sant',
       'Code_Wallon_de_lAction_sociale_et_de_la_Sant',
       'Code_Wallon_de_lAgriculture',
       'Code_Wallon_de_lEnseignement_Fondamental_et_de_lEnseignement_Secondaire',
       'Code_Wallon_de_lEnvironnement',
       'Code_Wallon_de_lHabitation_Durable',
       'Code_Wallon_du_Bien_tre_des_animaux',
       'Code_Wallon_du_Dveloppement_Territorial',
       'Code_dInstruction_Criminelle', 'Code_de_Droit_Economique',
       'Code_de_Droit_International_Priv',
       'Code_de_lEau_intgr_au_Code_Wallon_de_lEnvironnem

In [11]:
# create the partitions
codes = df_chunk_articles['normalized_cat'].unique()

for code in codes:
    client.create_partition(collection_name = collection_name,
                            partition_name=code)

### Encode chunked articles 

In [125]:
def encode_chunks(model, articles_list, batch_size=12, max_length=8192):
    encoded_vectors = []
    
    for i in tqdm(range(0, len(articles_list), batch_size), desc="Encoding chunks"):
        batch_articles = articles_list[i:i + batch_size]
        
        # Ensure all articles in the batch are strings
        # batch_articles = [str(article) for article in batch_articles]
        
        # Encode the batch
        try:
            batch_encoded = model.encode(
                batch_articles,
                batch_size=batch_size,
                max_length=max_length
            )["dense_vecs"]
            
            # Append the batch result to the list of encoded vectors
            encoded_vectors.append(batch_encoded)
        except Exception as e:
            print(f"Error encoding batch {i // batch_size + 1}: {e}")
    
    # Convert the list of encoded vectors to a single numpy array
    encoded_vectors = np.vstack(encoded_vectors)
    
    return encoded_vectors

In [135]:
# Example usage:

# List of articles
articles_list = df_chunk_articles['chunk'].tolist()

# Encode the list of articles
encoded_vectors = encode_chunks(bge_m3, articles_list)


Encoding chunks: 100%|██████████| 2772/2772 [07:21<00:00,  6.28it/s]


In [21]:
# Adjusted function to save embeddings
def save_embeddings(embeddings: np.ndarray, filepath: str):
    """Save embeddings to disk using numpy's compressed format"""
    np.savez_compressed(filepath, embeddings=embeddings)

In [136]:
# Optionally save to disk
save_embeddings(encoded_vectors, 'embeddings_RSchunks_bgem3.npz')

##### Insert the entities

In [12]:
def load_embeddings_as_dict(filepath: str, df_chunk_articles) -> Dict[int, np.ndarray]:
    """Load embeddings from disk and return as a dictionary of id:embedding pairs"""
    data = np.load(filepath)
    ids = df_chunk_articles['id_chunk'].tolist()
    embeddings = data['embeddings']
    
    embeddings_dict = {int(id_): emb for id_, emb in zip(ids, embeddings)}
    
    return embeddings_dict



In [13]:
embeddings_dict = load_embeddings_as_dict('embeddings_RSchunks_bgem3.npz', df_chunk_articles)

In [14]:
# Create the entities
def process_embeddings_in_chunks_and_partitions(
    df_chunk_articles: pd.DataFrame,
    loaded_embeddings: Dict[int, np.ndarray],
    chunk_size: int = 1000
) -> Dict[str, List[Dict[str, Any]]]:

    # Initialize dictionary for partitioned entities
    partitioned_entities = {}

    # Process data in chunks with progress bar
    with tqdm(total=len(df_chunk_articles), desc="Processing entities") as pbar:
        for chunk_start in range(0, len(df_chunk_articles), chunk_size):
            # Get chunk of dataframe
            chunk_end = min(chunk_start + chunk_size, len(df_chunk_articles))
            df_chunk = df_chunk_articles.iloc[chunk_start:chunk_end]

            # Process chunk
            for _, row in df_chunk.iterrows():
                partition = row['normalized_cat']
                embedding = loaded_embeddings.get(row['id_chunk'])
                if embedding is not None:
                    entity = {
                        "id": row['id_chunk'],
                        "article_id": row['article_id'],
                        "chunk_article": str(row['chunk']),
                        "reference": row['reference'],
                        "embedding_articles": np.array(embedding, dtype=np.float16)  # np.array of float16 to meet Milvus requirements
                    }
                    if partition not in partitioned_entities:
                        partitioned_entities[partition] = []
                    partitioned_entities[partition].append(entity)

            # Update progress bar
            pbar.update(len(df_chunk))

    print(f"Processed entities into {len(partitioned_entities)} partitions.")
    return partitioned_entities


In [15]:
partition_entities = process_embeddings_in_chunks_and_partitions(df_chunk_articles, embeddings_dict)

Processing entities: 100%|██████████| 33259/33259 [00:02<00:00, 12929.26it/s]

Processed entities into 34 partitions.


In [16]:
def insert_entities_in_chunks(client, collection_name, partition_entities, batch_size=1000):
    for partition, entities in partition_entities.items():
        print(f"Inserting entities into partition: {partition}")
        for i in range(0, len(entities), batch_size):
            batch = entities[i:i + batch_size]
            try:
                client.insert(data=batch, collection_name=collection_name, partition_name=partition)
            except Exception as e:
                print(f"Error during insertion for batch {i // batch_size + 1} in partition {partition}: {e}")



In [17]:
# Example usage
insert_entities_in_chunks(client, collection_name, partition_entities, batch_size=100)

Inserting entities into partition: Code_Judiciaire
Inserting entities into partition: Code_de_la_Dmocratie_Locale_et_de_la_Dcentralisation
Inserting entities into partition: Code_de_Droit_Economique
Inserting entities into partition: La_Constitution
Inserting entities into partition: Code_Civil
Inserting entities into partition: Code_de_la_Navigation
Inserting entities into partition: Code_Pnal_Militaire
Inserting entities into partition: Code_dInstruction_Criminelle
Inserting entities into partition: Code_Wallon_de_lAgriculture
Inserting entities into partition: Code_du_Bien_tre_au_Travail
Inserting entities into partition: Code_Rglementaire_Wallon_de_lAction_sociale_et_de_la_Sant
Inserting entities into partition: Code_Bruxellois_de_lAmnagement_du_Territoire
Inserting entities into partition: Code_Wallon_de_lAction_sociale_et_de_la_Sant
Inserting entities into partition: Code_de_la_Fonction_Publique_Wallonne
Inserting entities into partition: Code_Wallon_de_lEnseignement_Fondamental_

-------------

### Test search: Retrieve

In [23]:
res = client.list_partitions(collection_name=collection_name)
print(res)

['_default', 'Code_Judiciaire', 'Code_de_la_Dmocratie_Locale_et_de_la_Dcentralisation', 'Code_de_Droit_Economique', 'La_Constitution', 'Code_Civil', 'Code_de_la_Navigation', 'Code_Pnal_Militaire', 'Code_dInstruction_Criminelle', 'Code_Wallon_de_lAgriculture', 'Code_du_Bien_tre_au_Travail', 'Code_Rglementaire_Wallon_de_lAction_sociale_et_de_la_Sant', 'Code_Bruxellois_de_lAmnagement_du_Territoire', 'Code_Wallon_de_lAction_sociale_et_de_la_Sant', 'Code_de_la_Fonction_Publique_Wallonne', 'Code_Wallon_de_lEnseignement_Fondamental_et_de_lEnseignement_Secondaire', 'Code_des_Socits_et_des_Associations', 'Code_Wallon_du_Dveloppement_Territorial', 'Code_Pnal', 'Code_de_lEau_intgr_au_Code_Wallon_de_lEnvironnement', 'Code_Wallon_de_lEnvironnement', 'Code_Wallon_de_lHabitation_Durable', 'Code_Electoral', 'Code_Electoral_Communal_Bruxellois', 'Code_Consulaire', 'Code_Forestier', 'Code_Bruxellois_du_Logement', 'Code_Rural', 'Code_Ferroviaire', 'Codes_des_Droits_et_Taxes_Divers', 'Code_de_Droit_Intern

##### Embedding Model

In [18]:
bge_m3 = BGEM3FlagModel('BAAI/bge-m3',  
                       use_fp16=True, 
                       device='cuda')


Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

c:\Users\mthia\anaconda3\envs\AI_App\Lib\site-packages\FlagEmbedding\BGE_M3\modeling.py:335: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  colbert_state_dict = torch.load(os

In [19]:
def generate_embedding(article):
    embedding = bge_m3.encode([article], batch_size=12, max_length=1024)["dense_vecs"]
    return embedding[0]

In [20]:
df_questions = pd.read_csv('questions_train.csv')


df_questions['complet_quesiton'] = df_questions.apply(lambda row: row['question'] if pd.isna(row['extra_description']) else row['question'] + " " + row['extra_description'], axis=1)

In [21]:
m = 3
query_vectors = generate_embedding(df_questions['complet_quesiton'].iloc[m])

In [24]:
selected_partitions = res[1:]
client.load_partitions(collection_name = collection_name,
                                    partition_names= selected_partitions)

search_results = client.search(
    collection_name= collection_name,
    data = [query_vectors],  
    partition_names= selected_partitions,
    limit=20,  
    search_params={"metric_type": "COSINE", "params":{}},
    output_fields= ['id', 'article_id', 'chunk_article']
)

In [26]:
print(df_questions['article_ids'].iloc[m])
# formatted_result = json.dumps(search_results[0], indent=3, ensure_ascii=False)
# print(formatted_result)

predicted_ids = [result["entity"]["article_id"] for result in search_results[0]]
print(predicted_ids)

12012,12030,12031,12032,12033,12034,12035
[12155, 12062, 12031, 12025, 12025, 11258, 12043, 12042, 12041, 12153, 12108, 12054, 12124, 12050, 12156, 12092, 12153, 12032, 12153, 12050]


In [106]:
print(df_questions['article_ids'].iloc[m])
# formatted_result = json.dumps(search_results[0], indent=3, ensure_ascii=False)
# print(formatted_result)

predicted_ids = [result["entity"]["article_id"] for result in search_results[0]]
print(predicted_ids)

12012,12030,12031,12032,12033,12034,12035
[12155, 12062, 12031, 12025, 12108, 12153, 12054, 12124, 12050, 12156, 12153, 12032, 12050, 12056, 12081, 10410, 10829, 883, 12303, 12033]


In [80]:
print(df_questions['article_ids'].iloc[m])
formatted_result = json.dumps(search_results[0], indent=3, ensure_ascii=False)
print(formatted_result)

12012,12030,12031,12032,12033,12034,12035
[
   {
      "id": 15526,
      "distance": 0.66339510679245,
      "entity": {
         "id": 15526,
         "article_id": 12155,
         "chunk_article": "Tout pouvoir public relevant du champ d'activité de la société, tout centre d'insertion socioprofessionnelle agréé en vertu du décret du 10 juillet 2013 relatif aux centres d'insertion socioprofessionnelle ou tout organisme à finalité sociale, peut prendre en location un logement d'utilité publique, géré par une société de logement de service public afin de le mettre à disposition, sous sa seule responsabilité, d'un ménage de catégorie 1 et 2.Le nombre de logements pouvant être pris ainsi en location est limité à 5% du patrimoine de la société de logement de service public, parmi les logements déterminés par celle-ci, sur la base de critères objectifs dument motivés.Ce pourcentage ne tient pas compte des logements conventionnés dans le cadre d'un projet spécifique autorisés par la Société

-----------

In [124]:
entity_ids = [result["entity"]["article"] for result in search_results[0]]
print(entity_ids)

["La société doit procéder au recrutement d'un référent social chargé d'assurer le lien avec les acteurs sociaux locaux pour offrir un accompagnement social aux locataires de logements d'utilité publique gérés par la société, en veillant particulièrement à ce que l'accompagnement des personnes en transition entre les modes d'hébergement et le logement social accompagné soit assuré.Le Gouvernement détermine les conditions de recrutement du référent social ainsi que les modalités de mise en réseau de l'accompagnement social.Le Gouvernement subventionne la rémunération du référent social dans les conditions qu'il détermine.", "Le Gouvernementpeut accorder une aide à toute société de logement de service public qui acquiert un bâtiment non améliorable en vue de le démolir et d'affecter le terrain ainsi libéré à la construction de logements, et accessoirement, dans les limites fixées par le Gouvernement, à des équipements d'intérêt collectif en ce compris les éléments constitutifs d'un résea

#### ALL Predictions

In [28]:
def retrieve_all_predictions(df_questions, client, collection_name):
    predictions_dict = {}

    # all partitions
    all_partitions = client.list_partitions(collection_name=collection_name)

    for m in tqdm(range(len(df_questions))):
        # Generate the query vectors
        query_vectors = generate_embedding(df_questions['complet_quesiton'].iloc[m])
        
        # # Select and normalize partitions
        # selected_partitions = df_questions['big_category'].iloc[m]
        # normalized_selected_partitions = [re.sub(r'\s+|-', '_', partition) for partition in selected_partitions]
        # normalized_selected_partitions = [re.sub(r'[^a-zA-Z0-9_]', '', partition) for partition in normalized_selected_partitions]

        # Load partitions into the client
        client.load_partitions(collection_name=collection_name, partition_names= all_partitions[1:]) # remove the default partition
        
        # Search for results in the collection
        search_results = client.search(
            collection_name=collection_name,
            data=[query_vectors],
            partition_names= all_partitions[1:],
            limit=40,
            search_params={"metric_type": "COSINE", "params": {}},
            output_fields=['id', 'article_id', 'chunk_article']
        )
        
        # Collect predicted_ids and predictions_articles
        predicted_ids = [result["entity"]["article_id"] for result in search_results[0]]
        predictions_articles = [result["entity"]["chunk_article"] for result in search_results[0]]
        
        # Populate the dictionary
        predictions_dict[m] = {
            'predicted_ids': predicted_ids,
            'predictions_articles': predictions_articles
        }
    
    return predictions_dict


In [29]:
all_predicted_dict = retrieve_all_predictions(df_questions, client, collection_name)

100%|██████████| 886/886 [01:39<00:00,  8.91it/s]


In [85]:
all_predicted_dict[885]

{'predicted_ids': [2260,
  18873,
  19500,
  14583,
  2029,
  15821,
  1726,
  14411,
  2189,
  2160,
  2415,
  14503,
  2453,
  1443,
  1449,
  14502,
  2183,
  14598,
  2254,
  2434],
 'predictions_articles': ["a) céder ou donner en gage des créances hypothécaires;b) percevoir le prix de l'aliénation d'immeubles ou le remboursement de créances hypothécaires, donner mainlevée des inscriptions;c) accepter ou refuser un legs ou une donation lorsqu'il est stipulé que les biens légués ou donnés seront communs;d) contracter un emprunt;e) conclure un contrat de crédit, visé par la loi du 12 juin 1991 relative au crédit a la consommation, sauf si ces actes sont nécessaires aux besoins du ménage ou à l'éducation des enfants.",
  '2 Faute par le tiers détenteur de payer les dettes privilégiées et hypothécaires dans les termes et délais accordés au débiteur, ou de remplir les formalités qui seront établies ci-après pour purger sa propriété, chaque créancier a le droit de faire vendre le navire 

In [30]:
import gzip

# Store the result in a compressed JSON file
with gzip.open('all_predictions_RSChunk.json.gz', 'wt', encoding='utf-8') as f:
    json.dump(all_predicted_dict, f)

print('Predictions saved in compressed format!')


Predictions saved in compressed format!
